<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Exploratory-data-analysis" data-toc-modified-id="Exploratory-data-analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Exploratory data analysis</a></span><ul class="toc-item"><li><span><a href="#比赛介绍" data-toc-modified-id="比赛介绍-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>比赛介绍</a></span></li><li><span><a href="#相关有趣的知识点" data-toc-modified-id="相关有趣的知识点-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>相关有趣的知识点</a></span></li><li><span><a href="#读入数据" data-toc-modified-id="读入数据-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>读入数据</a></span><ul class="toc-item"><li><span><a href="#直接-pd.read_csv-读入" data-toc-modified-id="直接-pd.read_csv-读入-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>直接 <code>pd.read_csv</code> 读入</a></span></li><li><span><a href="#读入成一个-dataframe" data-toc-modified-id="读入成一个-dataframe-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>读入成一个 dataframe</a></span></li></ul></li><li><span><a href="#数据基本处理" data-toc-modified-id="数据基本处理-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>数据基本处理</a></span><ul class="toc-item"><li><span><a href="#理解每个变量的含义" data-toc-modified-id="理解每个变量的含义-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>理解每个变量的含义</a></span></li><li><span><a href="#Target-variable" data-toc-modified-id="Target-variable-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Target variable</a></span></li><li><span><a href="#full-Vistior-ld" data-toc-modified-id="full-Vistior-ld-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>full Vistior ld</a></span></li></ul></li><li><span><a href="#基本数据分析" data-toc-modified-id="基本数据分析-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>基本数据分析</a></span></li></ul></li></ul></div>

# Exploratory data analysis 

*By Jared Gong*

探索性数据分析总是第一步。我首先需要读入数据，分析几个重要的变量，做一个简单的模型。

## 比赛介绍

>> Google Analytics Customer Revenue Prediction 

**Objective of the competition:**

在本次竞赛中，我们面临的挑战是分析Google Merchandise Store（也称为GStore，销售谷歌周边）客户数据集，predicting the log of the total revenue per user, be aware that not all rows in `test.csv` will correspond to a row in the submission, but all unique `fullVisitorIds` will correspond to a row in the submission.


有多个列包含不同深度的JSON blob。在其中一个JSON列（`total`）中，子列 `transactionRevenue` 包含我们尝试预测的收入信息。该子列仅存在于训练数据中。具体的来说，我们预测每个用户的所有交易总和的自然对数。对于测试集中的每个用户，目标是：

$$
\text{target}^\text{user} = \ln \sum_{i=1}^n \text{transaction}_i^\text{user}
$$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## 相关有趣的知识点

一种格式化输出

In [ ]:
x = 2
f'{x} is a number'

In [ ]:
x = 2
'{} is a number'.format(x)

## 读入数据

本数据有多个列包含不同深度的JSON blob。

### 直接 `pd.read_csv` 读入

In [ ]:
tmp = pd.read_csv('../input/train.csv', nrows=2)
print(tmp.shape)
tmp

总共12个变量：

1. `channelGrouping`  - 用户通过其进入商店的渠道。
1. `date` - 用户访问Store的日期
1. `device` - 用户访问 Store 的设备类型
1. `fullVisitorId` - Google Merchandise Store的每个用户的唯一标识符。
1. `geoNetwork`  - 包含有关用户地理位置的信息。
1. `sessionId` - 此次访问商店的唯一标识符。
1. `socialEngagementType` - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
1. `totals` - This section contains aggregate values across the session.
1. `trafficSource` - This section contains information about the Traffic Source from which the session originated.
1. `visitId`  - 此会话的标识符。这是通常存储为_utmb cookie的值的一部分。这仅对用户而言是唯一的。对于完全唯一的ID，您应该使用fullVisitorId和visitId的组合。
1. `visitNumber`  - 此用户的会话号。如果这是第一个会话，则将其设置为1。
1. `visitStartTime`  - 时间戳（表示为POSIX时间）。

In [ ]:
tmp['device']

In [ ]:
# json_normalize(tmp['device']) # 此时会报错，因为读入的方式不对

下面讲解读入 `json` 文件！

In [ ]:
import json
from pandas.io.json import json_normalize

csv_path = '../input/train.csv'
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
nrows = 2 

df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, 
                     nrows=nrows)

In [ ]:
dc = json_normalize(df['device'])
dc.columns = [f"{column}.{subcolumn}" for subcolumn in dc.columns]
dc

### 读入成一个 dataframe


关于数据集：

与大多数其他的kaggle比赛类似，我们给出了两个数据集

    train.csv
    test.csv

数据集中的每一行都是对商店的一次访问。我们预测每个用户的所有交易总和的自然对数。

需要注意的是，有多个列包含不同深度的JSON blob。在其中一个JSON列（`total`）中，子列 `transactionRevenue` 包含我们尝试预测的收入信息。

In [ ]:
import os
import json
from pandas.io.json import json_normalize

def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, 
                     nrows=nrows) # Important!!
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../input"))

In [ ]:
train = load_df(nrows=100000)
test =load_df('../input/test.csv', nrows=100000)
train.head()

## 数据基本处理

### 理解每个变量的含义

总共12个变量：

1. `channelGrouping`  - 用户通过其进入商店的渠道。
1. `date` - 用户访问Store的日期
1. `device` - 用户访问 Store 的设备类型
1. `fullVisitorId` - Google Merchandise Store的每个用户的唯一标识符。
1. `geoNetwork`  - 包含有关用户地理位置的信息。
1. `sessionId` - 此次访问商店的唯一标识符。
1. `socialEngagementType` - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
1. `totals` - This section contains aggregate values across the session.
1. `trafficSource` - This section contains information about the Traffic Source from which the session originated.
1. `visitId`  - 此会话的标识符。这是通常存储为_utmb cookie的值的一部分。这仅对用户而言是唯一的。对于完全唯一的ID，您应该使用fullVisitorId和visitId的组合。
1. `visitNumber`  - 此用户的会话号。如果这是第一个会话，则将其设置为1。
1. `visitStartTime`  - 时间戳（表示为POSIX时间）。

其中的四个变量 `device, geoNetwork, totals, trafficSource` 进一步处理成多个变量。

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.tail()

总共有54个特征, 包括 'channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device.browser', 'device.browserSize', 'device.browserVersion',
       'device.deviceCategory', 'device.flashVersion', 'device.isMobile',
       'device.language', 'device.mobileDeviceBranding',
       'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName',
       'device.mobileDeviceModel', 'device.mobileInputSelector',
       'device.operatingSystem', 'device.operatingSystemVersion',
       'device.screenColors', 'device.screenResolution', 'geoNetwork.city',
       'geoNetwork.cityId', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'totals.transactionRevenue', 'totals.visits', 'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source' 

###  Target variable


我们预测每个用户的所有交易总和的自然对数。对于测试集中的每个用户，目标是：

$$
\text{target}^\text{user} = \ln \sum_{i=1}^n \text{transaction}_i^\text{user}
$$

看看进行散点图。

In [ ]:
np.log1p(0)

In [ ]:
train["totals.transactionRevenue"] = train["totals.transactionRevenue"].astype('float')
gdf = train.groupby("fullVisitorId")["totals.transactionRevenue"].sum().reset_index()

plt.figure(figsize=(6,4))
plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["totals.transactionRevenue"].values)))
plt.xlabel('index', fontsize=12)
plt.ylabel('TransactionRevenue', fontsize=12)
plt.show()

In [ ]:
gdf.describe()


从中可以看出一个重要的点：

*The 80/20 rule has proven true for many businesses–only a small percentage of customers produce most of the revenue. As such, marketing teams are challenged to make appropriate investments in promotional strategies.*

### full Vistior ld

与客户 Id 相关的变量包括 `fullVisitorId, sessionId, visitId, visitNumber`，一个 `fullVisitorId` 包含多个 `sessionId`, 而一个 `sessionId` 唯一对应一个 `visitId`，个数和 `visitNumber` 相关。

In [ ]:
ids = train[["fullVisitorId", "sessionId", "visitId", "visitNumber"]]
ids.sort_values(by='sessionId')
ids.isnull().sum()

In [ ]:
ids['sessionId'].unique().shape, ids.shape, ids['fullVisitorId'].unique().shape

In [ ]:
gong = ids.groupby(by='sessionId')['sessionId'].count()
gong1 = gong.sort_values(ascending=False)[:7].index.values.tolist()
gong2 = ids.loc[[i in gong1 for i in ids.sessionId], :].index
gong3 = train.loc[gong2].sort_values(by='sessionId')
gong3

In [ ]:
gong3.columns

In [ ]:
gong3['date'].unique().shape[0] == 1

In [ ]:
gong3[[c for c in gong3.columns if gong3[c].unique().shape[0] != 1]]

## 基本数据分析

In [ ]:
nzi = pd.notnull(train["totals.transactionRevenue"]).sum()
nzr = (gdf["totals.transactionRevenue"] > 0).sum()
print("Number of instances in train set with non-zero revenue : ", nzi, " and ratio is : ", nzi / train.shape[0])
print("Number of unique customers with non-zero revenue : ", nzr, "and the ratio is : ", nzr / gdf.shape[0])


访客人数和共同访客人数：

现在让我们看一下训练和测试集中的不同访客数量，以及共同的访客的数量。

In [ ]:
print("Number of unique visitors in train set : ",train.fullVisitorId.nunique(), " out of rows : ",train.shape[0])
print("Number of unique visitors in test set : ",test.fullVisitorId.nunique(), " out of rows : ",test.shape[0])
print("Number of common visitors in train and test set : ",len(set(train.fullVisitorId.unique()).intersection(set(test.fullVisitorId.unique())) ))

查看具备有常数值的变量。

In [ ]:
[c for c in train.columns if train[c].nunique()==1]

More to come ...

In [ ]:
 train.shape